<a href="https://colab.research.google.com/github/zi-onion/24-2-ESAA/blob/main/10_04_%EB%AC%B8%EC%84%9C_%EA%B5%B0%EC%A7%91%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파이썬 머신러닝 완벽 가이드 개정 2판 8장 p538~549

## 07 문서 군집화 소개와 실습(Opinion Review 데이터 세트)



### **문서 군집화 개념**
문서 군집화: 비슷한 텍스트 구성의 문서를 군집화 하는 것.

문서 군집화는 학습 데이터가 필요없는 비지도학습 기반으로 작동함.



###**Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기**
Opinion Review 데이터 세트: Tripadvisor（호텔）, Edmunds.com（자동차）
, Amazon.com（전자제품） 사이트에서 가져온 리뷰 문서

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 700)

path = '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1.0/topics'

all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = [ ]
opinion_text = []
# 개별 파일들의 파일명은 filename_list 리스트로 취합,
# 개별 파일들의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합
for file_ in all_files:
  # 개별 파일을 읽어서 DataFrame으로 생성
  df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')

  # 절대경로로 주어진 파일명을 가공. Linux에서 수행 시에는 아래 \\를 / 변경.
  # 맨 마지막 .data 확장자도 제거
  filename_ = file_.split('\\')[-1 ]
  filename = filename_.split('.')[0]

  # 파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용을 추가.
  filename_list.append(filename)
  opinion_text.append(df.to_string())

# 파일명 리스트와 파일 내용 리스트를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...
1,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...
2,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"A few other things I'd like to point out is that you must push the micro, sized right angle end of the ac adapter until it snaps in place or the battery may not charge .\n0 The full size right shift k..."
3,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery , and a 1 .\n0 Not to mention that as of now..."
4,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,Another feature on the 255w is a display of the posted speed limit on the road which you are currently on right above your current displayed speed .\n0 I found myself not even looking at my car speedometer as I could easily see my current speed and the speed limit of my route at a glance .\n1 ...


In [6]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    lemmer = nltk.stem.WordNetLemmatizer()
    return [lemmer.lemmatize(token) for token in tokens]
    # remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [7]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# TF-IDF형태로 피처 벡터화

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' ,\
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 칼럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

군집화 수행(K-means)

In [10]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 randim_state=0
km_cluster = KMeans(n_clusters=5,max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [12]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...,1
1,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...,1
2,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"A few other things I'd like to point out is that you must push the micro, sized right angle end of the ac adapter until it snaps in place or the battery may not charge .\n0 The full size right shift k...",1
3,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery , and a 1 .\n0 Not to mention that as of now...",1
4,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,Another feature on the 255w is a display of the posted speed limit on the road which you are currently on right above your current displayed speed .\n0 I found myself not even looking at my car speedometer as I could easily see my current speed and the speed limit of my route at a glance .\n1 ...,3


In [15]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
23,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"All in all, a normal chain hotel on a nice location , I will be back if I do not find anthing closer to Picadilly for a better price .\n0 ...",0
27,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"We arrived at 23,30 hours and they could not recommend a restaurant so we decided to go to Tesco, with very limited choices but when you are hingry you do not careNext day they rang the bell at 8,00 hours to clean the room, not being very nice being waken up so earlyEvery day they gave u...",0
35,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"The Swissotel is one of our favorite hotels in Chicago and the corner rooms have the most fantastic views in the city .\n0 The rooms look like they were just remodled and upgraded, there was an HD TV and a nice iHome docking station to put my iPod so I could set the alarm to wake up with my music instead of the radio .\n1 ...",0
37,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 The second room was smaller, with a very inconvenient bathroom layout, but at least it was quieter and we were able to sleep .\n1 ...",0
40,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Great Location , Nice Rooms , H...",0
41,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,Parking was expensive but I think this is common for San Fran .\n0 there is a fee for parking but well worth it seeing no where to park if you do have a car .\n1 ...,0
50,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n0 They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still able to get on there to check our flight to Vegas the next day .\n1 ...,0


-> Cluster #0: 호텔 리뷰

In [16]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...,1
26,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"If a case was included, as with the Kindle 1, that would have been reflected in a higher price .\n0 lower overall price, with nice leather cover .\n1 ...",1
25,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"In fact, the entire navigation structure has been completely revised , I'm still getting used to it but it's a huge step forward .\n0 ...",1
22,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"The Eee Super Hybrid Engine utility lets users overclock or underclock their Eee PC's to boost performance or provide better battery life depending on their immediate requirements .\n0 In Super Performance mode CPU, Z shows the bus speed to increase up to 169 .\n1 One...",1
21,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"After I plugged it in to my USB hub on my computer to charge the battery the charging cord design is very clever !\n0 After you have paged tru a 500, page book one, page, at, a, time to get from Chapter 2 to Chapter 15, see how excited you are about a low battery and all the time it took to get there !\n1 ...",1
20,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"It feels as easy to read as the K1 but doesn't seem any crisper to my eyes .\n0 the white is really GREY, and to avoid considerable eye, strain I had to refresh pages every other page .\n1 The dream has always been a portable electronic device that could hold a ton of reading material, automate subscriptions and fa...",1
16,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"I had to uninstall anti, virus and selected other programs, some of which did not have listings in the Programs and Features Control Panel section .\n0 This review briefly touches upon some of the key features and enhancements of Microsoft's latest OS .\n1 ...",1
29,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Being able to change the font sizes is awesome !\n0 For whatever reason, Amazon decided to make the Font on the Home Screen ...",1
14,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Windows 7 is quite simply faster, more stable, boots faster, goes to sleep faster, comes back from sleep faster, manages your files better and on top of that it's beautiful to look at and easy to use .\n0 , faster about 20% to 30% faster at running applications than my Vista , seriously\n1 ...",1
9,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"As always, the video screen is sharp and bright .\n0 2, inch screen and a glossy, polished aluminum finish that one CNET editor described as looking like a Christmas tree ornament .\n1 ...",1


-> Cluster #1: 포터블 전자기기 및 주요 구성요소 리뷰

In [17]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
19,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"First of all, the interior has way too many cheap plastic parts like the cheap plastic center piece that houses the clock .\n0 3 blown struts at 30,000 miles, interior trim coming loose and rattling squeaking, stains on paint, and bug splats taking paint off, premature uneven brake wear, on 3rd windsh...",2
30,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 Seats are fine, in fact of all the smaller sedans this is the most comfortable I found for the price as I am 6', 2 and 250# .\n1 Great gas mileage and comfortable on long trips ...",2
34,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 ...,2
38,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"It's quiet, get good gas mileage and looks clean inside and out .\n0 The mileage is great, and I've had to get used to stopping less for gas .\n1 Thought gas ...",2
39,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Very happy with my 08 Accord, performance is quite adequate it has nice looks and is a great long, distance cruiser .\n0 6, 4, 3 eco engine has poor performance and gas mileage of 22 highway .\n1 Overall performance is good but comfort level is poor .\n2 ...",2
44,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Drivers seat not comfortable, the car itself compared to other models of similar class .\n0 ...",2
45,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,I love the new body style and the interior is a simple pleasure except for the center dash .\n0 ...,2
46,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,I previously owned a Toyota 4Runner which had incredible build quality and reliability .\n0 I bought the Camry because of Toyota reliability and qua...,2
47,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"After slowing down, transmission has to be kicked to speed up .\n0 ...",2
49,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Front seats are very uncomfortable .\n0 No memory seats, no trip computer, can only display outside temp with trip odometer .\n1 ...",2


-> Cluster #2: 주로 차량용 네비게이션 리뷰로 구성(킨들 등 일부 존재)

In [18]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
4,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,Another feature on the 255w is a display of the posted speed limit on the road which you are currently on right above your current displayed speed .\n0 I found myself not even looking at my car speedometer as I could easily see my current speed and the speed limit of my route at a glance .\n1 ...,3
5,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"It's fast to acquire satellites .\n0 If you've ever had a Brand X GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1 ...",3
6,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,The voice prompts and maps are wonderful especially when driving after dark .\n0 I also thought the the voice prompts of the 750 where more pleasant sounding than the 255w's .\n1 ...,3
10,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"3 quot widescreen display was a bonus .\n0 This made for smoother graphics on the 255w of the vehicle moving along displayed roads, where the 750's display was more of a jerky movement .\n1 ...",3
12,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,", and is very, very accurate .\n0 but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1 This functi...",3
13,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Another thing to consider was that I paid $50 less for the 750 and it came with the FM transmitter cable and a USB cord to connect it to your computer for updates and downloads .\n0 update and reroute much _more_ quickly than my other GPS .\n1 UPDATE ON THIS , It finally turned out that to see the elevation contours at lowe...",3
15,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...,3
17,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,It is easy to read and when touching the screen it works great !\n0 and zoom out buttons on the 255w to the same side of the screen which makes it a bit easier .\n1 ...,3


-> Cluster #3: 대부분 호텔군집 +킨들리뷰 하나로 구성

In [19]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
18,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Great location for tube and we crammed in a fair amount of sightseeing in a short time .\n0 All in all, a normal chain hotel on a nice lo...",4
24,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,The room was packed to capacity with queues at the food buffets .\n0 The over zealous staff cleared our unfinished drinks while we were collecting cooked food and movement around the room with plates was difficult in the crowded circumstances .\n1 ...,4
28,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"not customer, oriented hotelvery low service levelboor reception\n0 The room was quiet, clean, the bed and pillows were comfortable, and the serv...",4
31,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"The staff at Swissotel were not particularly nice .\n0 Each time I waited at the counter for staff for several minutes and then was waved to the desk upon my turn with no hello or anything, or apology for waiting in line .\n1 ...",4
32,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,The food for our event was delicious .\n0 ...,4
36,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,Mediocre room and service for a very extravagant price .\n0 ...,4
42,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,Staff are friendl...,4
43,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Both of us having worked in tourism for over 14 years were very disappointed at the level of service provided by this gentleman .\n0 The service was good, very friendly staff and we loved the free wine reception each night .\n1 ...",4
48,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Good Value good location , ideal choice .\n0 Great Location , Nice Rooms , Helpless Concierge\n1 ...",4


-> Cluster #4: 자동차에 대한 리뷰

=> 전반적으로 군집개수가 많아서 세분화되어 군집화됨

중심 수를 5->3으로 낮추어 군집화 진행

In [21]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

# 소속 클러스터를 cl니ster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df ['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
50,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n0 They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still able to get on there to check our flight to Vegas the next day .\n1 ...,0
18,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"Great location for tube and we crammed in a fair amount of sightseeing in a short time .\n0 All in all, a normal chain hotel on a nice lo...",0
23,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"All in all, a normal chain hotel on a nice location , I will be back if I do not find anthing closer to Picadilly for a better price .\n0 ...",0
24,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,The room was packed to capacity with queues at the food buffets .\n0 The over zealous staff cleared our unfinished drinks while we were collecting cooked food and movement around the room with plates was difficult in the crowded circumstances .\n1 ...,0
27,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"We arrived at 23,30 hours and they could not recommend a restaurant so we decided to go to Tesco, with very limited choices but when you are hingry you do not careNext day they rang the bell at 8,00 hours to clean the room, not being very nice being waken up so earlyEvery day they gave u...",0
28,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"not customer, oriented hotelvery low service levelboor reception\n0 The room was quiet, clean, the bed and pillows were comfortable, and the serv...",0
31,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"The staff at Swissotel were not particularly nice .\n0 Each time I waited at the counter for staff for several minutes and then was waved to the desk upon my turn with no hello or anything, or apology for waiting in line .\n1 ...",0
35,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"The Swissotel is one of our favorite hotels in Chicago and the corner rooms have the most fantastic views in the city .\n0 The rooms look like they were just remodled and upgraded, there was an HD TV and a nice iHome docking station to put my iPod so I could set the alarm to wake up with my music instead of the radio .\n1 ...",0
32,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,The food for our event was delicious .\n0 ...,0
37,/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 The second room was smaller, with a very inconvenient bathroom layout, but at least it was quieter and we were able to sleep .\n1 ...",0


*   Cluster #0은 포터블 전자기기로만 구성
*   Cluster #1은 자동차 리뷰로만 구성
*  Cluster #2는 호텔 리뷰로만 구성



### **군집별 핵심 단어 추출하기**
clusters_centers_: KMeans 객체가 제공하는 각 군집을 구성하는 단어 피처가 군집의 중심(Centroid)을 기준으로 얼마나 가깝게
위치해 있는지에 대한 정보.


In [23]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape:', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape: (3, 4611)
[[0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


cluster_centers_는 (3, 4611)
-> 군집이 3개, word 피처가 4611개로 구성을 의미

cluster_centers_는 0에서 1 사이의 값을 가지며, 1에 가까울수록 중심과 가까움.

속성은 넘파이의 ndarray

In [44]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
  cluster_details={}

  # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값 반환
  # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
  centroid_feature_ordered_ind=cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num]={}
    cluster_details[cluster_num]['cluster']=cluster_num

    # cluster_centers_argsort()[:, ::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features=[feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
    top_feature_values=cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features']=top_features
    cluster_details[cluster_num]['top_features_value']=top_feature_values
    filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames=filenames.values.tolist()

    cluster_details[cluster_num]['filenames']=filenames

  return cluster_details


cluster_details의 개별 군집번호,
핵심 단어, 핵심단어 중심 위치 상댓값, 파일명 속성 값 정보를 좀 더 보기 좋게 표현하기 위한 함수

In [45]:
def print_cluster_details(clijster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('####### Cluster {0}',format(cluster_num))
    print('Top features：', cluster_detail['top_features' ])
    print('Reviews 파일명 ：', cluster_detail['filenames'][:7])
    print('===============================================')

In [46]:
feature_names = tfidf_vect.get_feature_names_out()

cluster_details = get_cluster_details(cluster_model = km_cluster,cluster_data=document_df,
                                      feature_names = feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

####### Cluster {0} 0
Top features： ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 ： ['/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1']
####### Cluster {0} 1
Top features： ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 ： ['/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset1', '/content/drive/MyDrive/2024/ESAA/Data/OpinosisDataset

각 군집별 핵심단어
- Cluster #0(포터블 전자제품): 'screen', 'battery', battery life'등
- Cluster #1(자동차): 'interior', 'seat','mileage','comfortable' 등
- Cluster #2(호텔): 'room','hotel', 'service','staff' 등
